#### Dati di implementazione
- Mappe di feature
- Dimensione della tabella di hash per ogni feature (forse si può dedurre)

#### Scissione
Un bucket che supera una certa taglia viene scisso in questo modo:
1. Viene trovata la feature tra le k disponibili che rappresenta meglio i valori nel bucket, cioè quella che assume i valori più disparati (per ora è scelta a caso)
2. Viene creata una tabella di hash di taglia pari alla dimensione fornita dall'utente (table_sizes)
3. Alla tabella è associata una funzione di hash appropriata
4. Tutti i valori del bucket sono spostati in questa tabella
5. Il bucket è eliminato e sostituito da un puntatore alla tabella

In [101]:
import random
from copy import copy

# Facciamo che ogni bucket ha la stessa dimensione massima
MAXBUCKETSIZE = 10

In [281]:
class NodeTable():
    # insertion_function combina estrazione della feature e hashing
    # Tree contiene un puntatore alla struttura HashTree per quando bisogna prelevare una feature (da rifare)
    def __init__(self, size, insertion_function, hashtree):
        self.size = size
        self.table = [[] for x in range(size)]
        self.insertion_function = insertion_function
        self.hashtree = hashtree
    
    def insert(self, element):
        print(element, self.size, self.insertion_function(element))
        # print(inspect.getsource(self.insertion_function))
        # Scegli la destinazione in base all'hash
        index = self.insertion_function(element)
        cell = self.table[index]
        
        # Devo distinguere se in una data posizione c'è un bucket (list)
        # o un puntatore ad un altro nodo (NodeTable)
        if type(cell) is list: # Bucket
            cell.append(element)
            
            # Scindi il bucket se è troppo grande
            if len(cell) > MAXBUCKETSIZE:
                self.scission(index)
        elif type(cell) is NodeTable: # Pointer
            # Ripeti ricorsivamente nel nodo figlio
            cell.insert(element)
        else:
            print("UNKNOWN TYPE")
            exit()
        

    def scission(self, index):
        print("Scissione all'indice {}".format(index))
        cell = copy(self.table[index])

        # Per ora prende una feature a caso
        feature_map, hash_f, table_size = self.hashtree.random_feature()
        insertion_function = lambda x: hash_f(feature_map(x))

        if len(self.table) == 1:
            # Caso in cui è presente solo la radice (da fare meglio, sostanzialmente ripete __init__)
            self.size = table_size
            self.table = [[] for x in range(table_size)]
            self.insertion_function = insertion_function
            
            for value in cell:
                self.insert(value)
        else:
            newChild = NodeTable(table_size, insertion_function, self.hashtree)
            for value in cell:
                newChild.insert(value)
                                
            self.table[index] = newChild
        print("Done rearranging")
            
    def __str__(self):
        return "NodeTable\nSize: {}\nTable: {}".format(self.size, self.table)
        
    def prettyPrint(self, depth=0):
        result = "{} NodeTable [{}] =>\n".format(depth, len(self.table))
        for cell in self.table:
            result += "--- " * (depth+1)
            if type(cell) is list:
                result += "[{}] => {}".format(len(cell), cell)
            elif type(cell) is NodeTable:
                result += "() => {}".format(cell.prettyPrint(depth + 1))
            else:
                print("UNKNOWN TYPE")
                exit()
            result += "\n"
        return result

In [282]:
class HashTree():
    def __init__(self, feature_maps, table_sizes):
        # Gli hash sono da fare meglio
        # (mappa feature, hash per la tabella, taglia della tabella)
        self.features = list(zip(feature_maps, [lambda x, size=size: hash(x) % size for size in table_sizes], table_sizes))
        
        # Numero di feature disponibili
        self.feature_amount = len(feature_maps)

        # Il primo nodo non cerca una feature particolare e utilizza un solo bucket
        self.tree = NodeTable(1, lambda x: 0, self)
        
    def insert(self, element):
        self.tree.insert(element)
        
    def random_feature(self):
        return self.features[random.randrange(self.feature_amount)]

    def __str__(self):
        return self.tree.prettyPrint()

### Test

In [283]:
FEATURE_AMOUNT=5
DATA_SIZE=3
DATA_AMOUNT=50

test_data = [[random.randint(1, 100) for y in range(DATA_SIZE)] for x in range(DATA_AMOUNT)]

feature_maps = [
    lambda x: x[1] * 2 + x[2],
    lambda x: x[0] ** 2,
    lambda x: x[2] + x[1] + x[0],
    lambda x: x[1] ** x[0],
    lambda x: x[2] + x[2] + x[1]
]
table_sizes = [random.randint(3, 10) for x in range(FEATURE_AMOUNT)]

ht = HashTree(feature_maps, table_sizes)

for test in test_data:
    ht.insert(test)

[23, 3, 5] 1 0
[29, 75, 39] 1 0
[84, 31, 52] 1 0
[27, 53, 4] 1 0
[41, 71, 89] 1 0
[99, 59, 80] 1 0
[19, 84, 18] 1 0
[42, 81, 38] 1 0
[42, 7, 72] 1 0
[92, 17, 79] 1 0
[63, 83, 93] 1 0
Scissione all'indice 0
[23, 3, 5] 7 6
[29, 75, 39] 7 6
[84, 31, 52] 7 2
[27, 53, 4] 7 5
[41, 71, 89] 7 4
[99, 59, 80] 7 2
[19, 84, 18] 7 1
[42, 81, 38] 7 3
[42, 7, 72] 7 4
[92, 17, 79] 7 0
[63, 83, 93] 7 3
Done rearranging
[66, 51, 53] 7 3
[97, 50, 5] 7 4
[28, 52, 75] 7 6
[90, 7, 15] 7 2
[53, 32, 12] 7 0
[71, 19, 76] 7 3
[64, 61, 74] 7 6
[31, 22, 78] 7 3
[24, 97, 73] 7 5
[24, 86, 15] 7 4
[15, 73, 87] 7 2
[93, 63, 72] 7 4
[47, 86, 34] 7 0
[85, 15, 71] 7 3
[100, 17, 47] 7 6
[55, 26, 35] 7 5
[48, 63, 46] 7 1
[98, 51, 8] 7 4
[7, 57, 88] 7 2
[45, 66, 99] 7 5
[61, 47, 29] 7 0
[31, 17, 20] 7 1
[7, 6, 11] 7 0
[70, 47, 15] 7 0
[66, 36, 12] 7 4
[19, 6, 67] 7 0
[31, 91, 7] 7 0
[63, 29, 71] 7 3
[35, 26, 89] 7 1
[61, 59, 29] 7 5
[4, 7, 98] 7 0
[83, 48, 85] 7 1
[94, 27, 38] 7 5
[50, 10, 81] 7 4
[84, 62, 23] 7 3
[39, 28,

In [284]:
print(ht)

0 NodeTable [7] =>
--- [10] => [[92, 17, 79], [53, 32, 12], [47, 86, 34], [61, 47, 29], [7, 6, 11], [70, 47, 15], [19, 6, 67], [31, 91, 7], [4, 7, 98], [9, 51, 83]]
--- [6] => [[19, 84, 18], [48, 63, 46], [31, 17, 20], [35, 26, 89], [83, 48, 85], [42, 80, 34]]
--- [5] => [[84, 31, 52], [99, 59, 80], [90, 7, 15], [15, 73, 87], [7, 57, 88]]
--- [9] => [[42, 81, 38], [63, 83, 93], [66, 51, 53], [71, 19, 76], [31, 22, 78], [85, 15, 71], [63, 29, 71], [84, 62, 23], [39, 28, 82]]
--- [8] => [[41, 71, 89], [42, 7, 72], [97, 50, 5], [24, 86, 15], [93, 63, 72], [98, 51, 8], [66, 36, 12], [50, 10, 81]]
--- [6] => [[27, 53, 4], [24, 97, 73], [55, 26, 35], [45, 66, 99], [61, 59, 29], [94, 27, 38]]
--- [6] => [[23, 3, 5], [29, 75, 39], [28, 52, 75], [64, 61, 74], [100, 17, 47], [12, 18, 15]]

